## Additive Utilities

In [2]:
import numpy as np

In [ ]:
def Citeration_additive(C,f,alpha,n,p,K,iprint):
    vstore= np.zeros((n*K,1))
    v= np.zeros((n,1))

    for k in range(0,K):
        dv= np.zeros((n,p))
        # Calculate and store subdomain contributions (independently)
        for i in range(0,p):
            CC= C[i,0:n,0:n]
            ff= f[i,0:n]
#             print("top",np.shape(ff))
            ff=ff[np.newaxis]
#             print(np.shape(ff))
#             print(np.shape(CC))
            dv[0:n,i]= ff - CC @ v[0:n,0]
            if iprint >= 6:
                print(f'Citeration_additive: dv on subdomain {i:3d} at iteration {k:3d}')
                print(dv.T)

        # Perform update by summing weighted subdomain contributions
        for i in range(p):
            v[0:n,0]= v[0:n,0] + alpha * dv[0:n,i]

        if iprint >= 2:
            print('Outer iteration', k)
            print(v.T)
    
        # Store iterates in a 1D vector
        vstore[k*n:(k + 1) * (n)]= v[range(n)]

    return vstore

In [2]:
def Dmatrix_additive(C,alpha,n,p,iprint):
    D= np.zeros((n,n))
    for i in range(p):
        CC= C[i,0:n,0:n]
        D= D + CC

    D= np.identity(n) - alpha * D
    if iprint >= 4:
        print('dmatrix_additive:  matrix')
        print(np.round(D,4))
    return D

In [1]:
def gvector_additive(f,alpha,n,p,iprint):
    # Construct g vector for additive Schwarz

    g= np.zeros((n,1))
    for i in range(p):
        ff= f[i]
        ff= ff.reshape((n,1))
        g= g + ff
    g[0:n,0]= alpha * g[0:n,0]

    if iprint >= 4:
        print('Print g vector')
        print(g)
    return g

In [13]:
def Diteration_additive(D,g,n,p,K,iprint):
    wstore= np.zeros((n*K,1))
    rng= np.random.default_rng((n,1))
    w= rng.random((n,1))

    # Perform K iterations
    for k in range(K):
        w= D @ w + g
        wstore[k*n:(k+1) * n]= w[range(n)]
        if iprint >= 4:
            print('Diteration_additive: w at iteration ', k)
            print(w)
    print(f'norm(D)= {np.linalg.norm(D):13.6e}')
    print(f'Asymptotic solution: z= inv(I-D)@g')
    z= np.linalg.inv(np.eye(n) - D) @ g
    print(f'K= {K:5d}, norm(w-z)= {np.linalg.norm(w - z):13.6e}')

    return wstore

In [ ]:
def Umatrix_additive(D,n,K,iprint):
    # Construct U matrix for additive Schwarz

    U= np.identity(K*n)

    for i in range(K-1):
        U[(i+1) * n:(i+2) * n, i * n:(i+1) * n]= -D

    if iprint >= 4:
        print('U matrix')
        print(np.round(U,1))
    return U

In [ ]:
def hvector_additive(g,n,K,iprint):
    #Construct h vector for additive Schwarz

    h= np.zeros((K*n,1))
    for i in range(K):
        h[i * n:(i+1) * n,0:1]= g  ##should apply the 0:1 to other functions with same problem
    if iprint >= 4:
        print('h vector')
        print(h)
    return h

In [1]:
def Uhsolve_additive(U,h,n,K,iprint):
    # Perform additive Schwarz iterations
    # K iterations performed by single matrix solve
    # Store solutions from K iterations in a one-dimensional vector

    z= np.linalg.solve(U,h)
    zz=np.zeros((n,K))
    if iprint > 0:
        for k in range(K):
            print('Uhsolve_additive: z at iteration', k)
            zz[0:n,k]= z[k * n:(k+1) * n, 0]
            print(zz[0:n,k])
    return z

In [2]:
def Citeration_additive_approx(C,f,alpha,n,p,K,mpert,wpert,iprint):
    # Perform additive Schwarz iteration
    # Compute contribution from each subdomain independently
    # Introduce random error in each subdomain computation
  
    # Store exact solutions from K iterations in a one-dimensional vector
    # Store approximate solutions from K iterations in a one-dimensional vector
    # Store residuals from K iterations in a one-dimensional vector

    CC= np.zeros((n,n))

    v= np.zeros((n,p))
    va= np.zeros((n,p))
    r= np.zeros((n,p))

    vglobal= np.zeros((n,1))
    vaglobal= np.zeros((n,1))
    rglobal_store= np.zeros((K*n,1))

    vglobal_store=np.zeros((n*K,1))
    vexact_store= np.zeros((n*(K+1),1))
    vaglobal_store= np.zeros((n*(K+1),1))

    for k in range(K):
        if iprint >= 2:
            print('Citeration_additive_approx: Outer iteration', k)

        for i in range(p):
            # Compute the contributions from each subdomain
            CC[0:n,0:n]= C[i]
            ff= f[i].reshape(n,1)
            v[0:n,i:i+1]= ff - CC @ vglobal[0:n,0:1]
            index= k*p*n + i*n
            va[0:n,i:i+1]= ff - CC @ vaglobal[0:n,0:1] + mpert * wpert[index: index + n]

        # Perform update of global solution
        for i in range(p):
            vglobal[0:n,0:1]= vglobal[0:n,0:1] + alpha*v[0:n,i:i+1]
            vaglobal[0:n,0:1]= vaglobal[0:n,0:1] + alpha*va[0:n,i:i+1]

        vglobal_store[k * n:(k+1) * n,0:1]= vglobal
        vaglobal_store[(k+1) * n:(k+2) * n,0:1]= vaglobal

        diff= vglobal - vaglobal
        if iprint >= 2:
            print(f'Difference = {np.linalg.norm(diff):10.4e}')

        D= Dmatrix_additive(C,alpha,n,p,iprint)
        g= gvector_additive(f,alpha,n,p,iprint)
        rglobal= g + D @ vaglobal_store[k * n:(k+1) * n,0:1] - vaglobal[0:n,0:1]
        rglobal_store[k * n:(k+1) * n,0]= rglobal[0:n,0]
  
    return vglobal_store,vaglobal_store,rglobal_store

In [1]:
def Citeration_additive_adjoint(C,psi,alpha,n,p,K,iprint):
#
# Perform additive Schwarz iteration
# Compute contribution from each subdomain independently
# Introduce random error in each subdomain computation
#
# Store exact solutions from K iterations in a one-dimensional vector
# Store approximate solutions from K iterations in a one-dimensional vector
# Store residuals from K iterations in a one-dimensional vector
#

    CC= np.zeros((n,n))
    phi= np.zeros((n,1))

    phiglobal= np.zeros((n,1))
    phiglobal_store= np.zeros((K * n,1))

    phiglobal= np.copy(psi)
    phiglobal_store[(K-1) * n:K * n,0]= phiglobal[0:n,0]
    
    phi= np.zeros((n,p))
    print(n,p,K)
    for k in range(K-1,0,-1):

        for i in range(p-1,-1,-1):
            # Compute the contributions from each subdomain
            CC=C[i,:,:]
            phi[:,i]= (CC.T @ phiglobal).reshape(-1) #make it 1 dimenstional

            if iprint >= 4:
                print(f'Citeration_additive_adjoint: k= {k:1d}, i= {i:1d}')
                tmp= phi[:,i]
                print(tmp.T)

        # Perform update
        sum_phi= np.zeros((n,1))
        for i in range(p):
            sum_phi= sum_phi + (phi[:,i]).reshape(-1,1)

        phiglobal= phiglobal - alpha * sum_phi
        phiglobal_store[(k-1) * n : k * n, 0]= phiglobal.reshape(-1)
    return phiglobal_store

In [24]:
def Diteration_additive_approx(D,g,alpha,n,p,K,wpert,mpert,iprint):
    #
    # Perform additive Schwarz iteration
    # Compute contribution from each subdomain independently
    # Introduce random error in each subdomain computation
    #
    # Store exact solutions from K iterations in a one-dimensional vector
    # Store approximate solutions from K iterations in a one-dimensional vector
    # Store residuals from K iterations in a one-dimensional vector
    #

    w= np.zeros((n,1))
    wa= np.zeros((n,1))
    wr= np.zeros((n,1))

    w_store= np.zeros((n*K,1))
    wa_store= np.zeros((n*(K+1),1))
    wr_store= np.zeros((n*K,1))

    kvpert= np.zeros((n*K,1))


    for i in range(0,K):
        i1= i * n
        i2= (i + 1)*n
        for j in range(p):
            j1= i*n*p + (j) * n
            j2= i*n*p + (j+1) * n
            kvpert[i1:i2]= kvpert[i1:i2] + alpha*wpert[j1:j2]
            #print('i= %4i, j= %4i, i1= %4i, i2= %4i, j1= %4i, j1= %4i \n', i, j, i1, i2, j1,j2)
            #print('mean(wpert(j1:j2)) = %13.6e \n', mean(wpert(j1:j2)))

        w= D @ w + g
        wa= D @ wa + g + mpert * kvpert[i1:i2]
        wr= (g.reshape(-1) + D @ wa_store[i*n:(i+1)*n,0]) - wa[0:n,0]

        w_store [i*n:(i+1)*n,0]= w.reshape(-1)
        wa_store[(i+1)*n:(i+2)*n,0]= wa.reshape(-1)
        wr_store[i*n:(i+1)*n,0]= wr.reshape(-1)
    return w_store,wa_store,wr_store

In [2]:
def Uhsolve_additive_approx(U,h,n,p,K,alpha,wpert,mpert,iprint):
    
    #  Perform multiplicative Schwarz iterations
    #  K iterations performed by single matrix solve
    #  Introduce random error
    #  Store solutions from K iterations in a one-dimensional vector
    # 
    #  Store exact solutions from K iterations in a one-dimensional vector
    #  Store approximate solutions from K iterations in a one-dimensional vector
    #  Store residuals from K iterations in a one-dimensional vector
    
    zexact=np.linalg.solve(U,h)
    zpert=np.zeros((n*K,1))
    
    for i in range(K):
        i1= i * n
        i2= (i+1) * n
        for j in range(p):
            j1= i*n*p + j * n
            j2= i*n*p + (j+1)*n
            zpert[i1:i2]= zpert[i1:i2] + alpha*mpert*wpert[j1:j2]
            
    zapprox= zexact + zpert
    
    zresid= h - U @ zapprox
    
    return zexact,zapprox,zresid